1. 使用前可能需要更新selenium对应Chrome版本的插件: https://chromedriver.storage.googleapis.com/index.html  
2. 你可以先跑一下前几个代码块，看错误提示是哪个Chrome版本，高于当前版本即可
3. 然后将解压得到的chromedriver.exe移动到chrome.exe所在的目录，一般来说（例如本例）该位置是C:\Program Files (x86)\Google\Chrome\Application  
4. （如电脑曾使用过selenium则不需此步）最后，配置环境变量，右键此电脑（注意不是快捷方式）-属性-高级系统设置-环境变量-系统环境变量-Path-编辑-新建，添加好上条的目录位置
5. 参考教程: https://blog.csdn.net/orange_xiang/article/details/82924296

In [ ]:
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import requests

from tqdm import tqdm

import pandas as pd 
#import numpy as np
#import math
import os

HOME = './'
FOLDER_NAME = 'Foreclosure_Reports_Ali'

In [ ]:
FOLDER_PATH = os.path.join(HOME,FOLDER_NAME)
os.makedirs(FOLDER_PATH,exist_ok=True)

In [ ]:
# 用selenium收集房源标题以及报告附件的下载链接
driver = webdriver.Chrome()
driver.implicitly_wait(10)

In [ ]:
# 打开网站
driver.get("https://sf.taobao.com/item_list.htm?spm=a213w.3064813.a214dqe.18.38e95162R0zyog&city=&province=%B1%B1%BE%A9&auction_start_seg=-1")
sleep(1)
# 勾选限制条件：住宅用房
driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/ul[1]/li/div[2]/ul/li[1]/em/a').click()
sleep(1)

In [ ]:
# 按拍卖状态：正在进行，即将开始，已结束，中止，撤回

In [ ]:
%%time


url_list = []
n_page = 1
while True:
    entry_list = driver.find_elements(By.CLASS_NAME,'goods-container')
    print('page {}: find {} entries.'.format(n_page,len(entry_list)))
    for entry in entry_list:
        url = entry.get_attribute('href')
        #print(url)
        url_list.append(url)
        
    # to next page
    next_page = driver.find_elements(By.CLASS_NAME,'ui-pager-next')
    if next_page:
        n_page += 1
        next_page[0].click()  # 必须保持浏览器页面正常大小，否则报错，因为是按照位置来点击的，模拟人的鼠标操作
        sleep(1)
    else:
        break

In [ ]:
len(url_list)

In [ ]:
pd.Series(url_list).to_csv('entry_url_list.csv')

In [ ]:
i = 0

In [ ]:
# in case of skipping a entry
i += 1

In [ ]:
# 用requests下载文件到指定文件夹
# 网页上的文件名(report_name)自跟文件类型后缀

# if exceptions, run this cell again
# to skip the current entry, run the above cell and then this cell

for url in tqdm(url_list[i:]):
#for url in ['https://paimai.jd.com/287456648']:
    driver.get(url)
    title = driver.find_elements(By.CLASS_NAME,'pm-name')
    if title:
        title = title[0].text
        #print(title)
        attachFiles = driver.find_elements(By.CLASS_NAME,'attachFiles')
        if attachFiles:
            attachFiles = attachFiles[0]
            files = attachFiles.find_elements(By.CLASS_NAME,'file')
            if files:
                print(title,url)
                for f in files:
                    report = f.find_element(By.XPATH,'a[1]')
                    report_name = report.text
                    report_url = report.get_attribute('href')
                    print(report_name,report_url)
                    entry_path = os.path.join(FOLDER_PATH,title)
                    os.makedirs(entry_path,exist_ok=True)
                    download_path = os.path.join(entry_path,report_name)
                    
                    r = requests.get(report_url)
                    with open(download_path, 'wb') as file:
                        file.write(r.content)
    i += 1

In [ ]:
i